#Inferring

The tasks where the model takes a text as input and
performs some kind of analysis. So this could be extracting labels,
extracting names, kind of understanding the
sentiment of a text, that kind of thing.
So if you want to extract a sentiment, positive or negative,
of a piece of text, in the traditional
machine learning workflow, you'd have to collect the label data set, train
a model, figure out how to deploy the model somewhere in
the cloud and make inferences. And that could work pretty well, but
it was, you know, just a lot of work to
go through that process. And also for every task, such
as sentiment versus extracting names versus
something else, you have to train and
deploy a separate model. One of the really nice
things about large language model is that for
many tasks like these, you can just write a
prompt and have it start generating results pretty
much right away. And that gives tremendous speed in terms
of application development. And you can also just use one model, one
API to do many different tasks rather than
needing to figure out how to train and deploy a lot of
different models.

In [1]:
!pip install openai

In [14]:
from openai import OpenAI
import os

In [13]:
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('openai')

In [21]:
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

In [57]:
def get_completion(prompt, model = 'gpt-3.5-turbo'):
  messages = [{'role':'user', 'content':prompt}]
  response = client.chat.completions.create(
      model = model,
      messages = messages,
      temperature = 0
  )
  return response.choices[0].message.content

## Product review text

In [78]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

## Sentiment (positive/negative)

In [79]:
prompt = f"""
What is the sentiment of the following product review,
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of the review is positive. The reviewer is satisfied with the lamp, the customer service, and the company in general.


In [81]:
prompt = f"""
What is the sentiment of the following product review,
which is delimited with triple backticks?

Give your answer as a single word, either "positive"
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Positive


## Identify types of emotions

In [82]:
prompt = f"""
Identify a list of emotions that the writer of the
following review is expressing. Include no more than
five items in the list. Format your answer as a list of
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

happy, satisfied, grateful, impressed, content


## Extract product and company name from customer reviews

In [85]:
prompt = f"""
Identify the following items from the review text:
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks.
Format your response as a JSON object with
"Item" and "Brand" as the keys.
If the information isn't present, use "unknown"
as the value.
Make your response as short as possible.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Item": "lamp",
  "Brand": "Lumina"
}


## Do multiple tasks at once

If you are trying to
summarize many reviews from an online shopping e-commerce website,
it might be useful for your large collection of reviews
to figure out what were the items, who made
the item, figure out positive and negative
sentiment, to track trends about positive or negative sentiment
for specific items or for specific
manufacturers. And in this example, we can ask it to format your response as a JSON object with "Item" and "Brand" as
the keys. And so if we do that, it says the
item is a lamp, the brand is Lumina, and you can easily load this
into the Python dictionary to then do additional processing
on this output.

In [86]:
prompt = f"""
Identify the following items from the review text:
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks.
Format your response as a JSON object with
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown"
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
    "Sentiment": "positive",
    "Anger": false,
    "Item": "lamp",
    "Brand": "Lumina"
}


##Inferring topics

In [87]:
story = """
In a recent survey conducted by the government,
public sector employees were asked to rate their level
of satisfaction with the department they work at.
The results revealed that NASA was the most popular
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings,
stating, "I'm not surprised that NASA came out on top.
It's a great place to work with amazing people and
incredible opportunities. I'm proud to be a part of
such an innovative organization."

The results were also welcomed by NASA's management team,
with Director Tom Johnson stating, "We are thrilled to
hear that our employees are satisfied with their work at NASA.
We have a talented and dedicated team who work tirelessly
to achieve our goals, and it's fantastic to see that their
hard work is paying off."

The survey also revealed that the
Social Security Administration had the lowest satisfaction
rating, with only 45% of employees indicating they were
satisfied with their job. The government has pledged to
address the concerns raised by employees in the survey and
work towards improving job satisfaction across all departments.
"""

## Infer 5 topics

In [93]:
prompt = f"""
Determine five topics that are being discussed in the
following text, which is delimited by triple backticks.

Make each item one or two words long.

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

1. Survey
2. Job satisfaction
3. NASA
4. Social Security Administration
5. Government pledge


In [94]:
topic_list = [
    "nasa", "local government", "engineering",
    "employee satisfaction", "federal government"
]

## Make a news alert for certain topics

In [99]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as follows:
item from the list: 0 or 1

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

nasa: 1
local government: 0
engineering: 0
employee satisfaction: 1
federal government: 1



In machine learning, this is sometimes called a "Zero-Shot Learning Algorithm",
because we didn't give it any training data that was
labeled, so that's Zero-Shot. And with just a prompt, it
was able to determine which of these topics are covered in that news article.

In [100]:
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}
if topic_dict['nasa'] == 1:
    print("ALERT: New NASA story!")

ALERT: New NASA story!
